In [306]:
%load_ext autoreload
%autoreload 2
import blackhkjc as bh
import pandas as pd
import re
df = pd.read_pickle('handi.pkl')
df = df[df.HOME_TREND.apply(lambda x:len(x)) >=5]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [307]:
len(df)

64170

# Loop pattern 5,6,7,8

In [159]:
df_pat = df.copy()

In [160]:
df_pat = df.copy()
number_repeating_pattern = 5
#pattern = r'0-{' + str(number_repeating_pattern) + r'}'
pattern = r'0-----+$'
df_pat['A_PAT_FOUND'] = df_pat.AWAY_TREND.apply(lambda x: 'Y' if re.findall(pattern,x) else 'N')
df_pat['H_PAT_FOUND'] = df_pat.HOME_TREND.apply(lambda x: 'Y' if re.findall(pattern,x) else 'N')
df_pat['PAT_FOUND'] = df_pat.apply(lambda rec: 'Y' if ((rec['A_PAT_FOUND'] =='Y') or (rec['H_PAT_FOUND']=='Y')) else 'N',axis=1)

# Generate Summary
df_summary = df_pat[df_pat['PAT_FOUND'] == 'Y']

# Generate Summary
df_found_init = df_summary.copy()
#df_found_init

In [162]:
summary_array = []

for number_repeating_patterns in [5,6,7,8]:
    df_summary = df_found_init.copy()
    pattern = r'0-{' + str(number_repeating_patterns) + r'}'
    df_summary['A_PAT_FOUND'] = df_summary.AWAY_TREND.apply(lambda x: 'Y' if re.findall(pattern,x) else 'N')
    df_summary['H_PAT_FOUND'] = df_summary.HOME_TREND.apply(lambda x: 'Y' if re.findall(pattern,x) else 'N')
    df_summary['PAT_FOUND'] = df_summary.apply(lambda rec: 'Y' if ((rec['A_PAT_FOUND'] =='Y') or (rec['H_PAT_FOUND']=='Y')) else 'N',axis=1)

    # Generate Summary
    df_found = df_summary[df_summary['PAT_FOUND'] == 'Y']

    print('number_repeating_patterns:', number_repeating_patterns)

    for k,v in sorted(bh.line_mapping.items(),key=lambda x:x[1]):
        #df_temp = df_found[(df_found.O_FIRST_LINE == v)].groupby(['HANDI_LABEL','H_PAT_FOUND','LAST_LINE']).agg({"ID": pd.Series.nunique}).reset_index()
        # Use last 10
        df_10 = df_found[(df_found.O_FIRST_LINE == v)].groupby('ID').apply(lambda x:x.iloc[0]).reset_index(drop=True).iloc[-10:]
        print(df_10.columns)
        df_temp = df_10.groupby(['HANDI_LABEL','H_PAT_FOUND','LAST_LINE']).agg({"ID": pd.Series.nunique}).reset_index()
        
        away_total = home_total = 0
        if df_temp.empty:
            home_sample = home_win_ratio = away_sample = away_win_ratio = 0
        else:
            LAST_LINE_LIST = df_temp.LAST_LINE.unique()
            for v_last_line in LAST_LINE_LIST:
                df_N = df_temp[(df_temp['H_PAT_FOUND'] == 'N') & (df_temp['HANDI_LABEL'] != 'D')&
                          (df_temp['LAST_LINE'] == v_last_line )]
                away_total = df_N.ID.sum()
                away_win_ratio, away_sample = bh.get_win_ratio_sample(df_N, venue='A')
                
                df_Y = df_temp[(df_temp['H_PAT_FOUND'] == 'Y') & (df_temp['HANDI_LABEL'] != 'D')&
                          (df_temp['LAST_LINE'] == v_last_line )]
                home_total = df_Y.ID.sum()
                home_win_ratio, home_sample = bh.get_win_ratio_sample(df_Y, venue='H')

                home_suggest_hands, away_suggest_hands = bh.home_away_suggest(home_win_ratio,away_win_ratio)
        
                summary_array.append(
                    (number_repeating_patterns,
                     v,
                     k,
                     v_last_line,
                     home_win_ratio,
                     home_sample,
                     home_total,
                     away_win_ratio,
                     away_sample,
                     away_total,
                     home_suggest_hands,
                     away_suggest_hands
                    )
                )

number_repeating_patterns: 5
Index([], dtype='object')


KeyError: 'HANDI_LABEL'

In [136]:
df_sum = pd.DataFrame(summary_array)

In [137]:
df_sum.columns = ['NO_REPEAT_PATTERN', 'F_LINE_ID','F_LINE','L_LINE',
                  'HOME_WIN_RATIO','HOME_SAMPLE',
                 'HOME_TOTAL','AWAY_WIN_RATIO','AWAY_SAMPLE','AWAY_TOTAL','HOME_SUGG', 'AWAY_SUGG'
                 ]

In [138]:
df_sum2 = df_sum[['F_LINE_ID','F_LINE','L_LINE','NO_REPEAT_PATTERN','HOME_WIN_RATIO',
                  'AWAY_WIN_RATIO','HOME_SAMPLE',
                 'AWAY_SAMPLE'
                 ]]

In [139]:
table = pd.pivot_table(df_sum2, index=['F_LINE_ID','F_LINE','L_LINE'], columns='NO_REPEAT_PATTERN', 
                       values=['HOME_WIN_RATIO','AWAY_WIN_RATIO','HOME_SAMPLE','AWAY_SAMPLE'],fill_value=0)

In [140]:
table.reset_index(col_level=1).to_csv('output.csv',index=False)

In [377]:
df_sum.head()

,NO_REPEAT_PATTERN,F_LINE_ID,F_LINE,L_LINE,HOME_WIN_RATIO,HOME_SAMPLE,HOME_TOTAL,AWAY_WIN_RATIO,AWAY_SAMPLE,AWAY_TOTAL,HOME_SUGG,AWAY_SUGG
0,5,0,(+2/+2.5),(+2),0.0,0,1,0.0,0,0,,
1,5,1,(+2),(+2/+2.5),0.0,0,0,0.0,0,1,,
2,5,1,(+2),(+2),1.0,1,1,0.0,0,0,"HOME, 4 hands",
3,5,2,(+1.5/+2),(+1.5/+2),0.0,0,2,0.5,1,2,,
4,5,2,(+1.5/+2),(+1/+1.5),0.0,0,2,0.0,0,0,,


In [173]:
df_sum = pd.DataFrame(summary_array)

In [174]:
df_sum.columns = ['NO_REPEAT_PATTERN', 'LINE_ID','LINE','HOME_WIN_RATIO','HOME_SAMPLE',
                 'HOME_TOTAL','AWAY_WIN_RATIO','AWAY_SAMPLE','AWAY_TOTAL','HOME_SUGG', 'AWAY_SUGG'
                 ]

In [287]:
df_sum.head()

,NO_REPEAT_PATTERN,F_LINE_ID,F_LINE,L_LINE,HOME_WIN_RATIO,HOME_SAMPLE,HOME_TOTAL,AWAY_WIN_RATIO,AWAY_SAMPLE,AWAY_TOTAL,HOME_SUGG,AWAY_SUGG
0,5,0,(+2/+2.5),(+2),0.0,0,1,0.0,0,0,,
1,5,1,(+2),(+2/+2.5),0.0,0,0,0.0,0,1,,
2,5,1,(+2),(+2),1.0,1,1,0.0,0,0,"HOME, 4 hands",
3,5,2,(+1.5/+2),(+1.5/+2),0.0,0,2,0.5,1,2,,
4,5,2,(+1.5/+2),(+1/+1.5),0.0,0,2,0.0,0,0,,


In [354]:
df_sum2 = df_sum[['F_LINE_ID','F_LINE','L_LINE','NO_REPEAT_PATTERN','HOME_WIN_RATIO',
                  'AWAY_WIN_RATIO','HOME_SAMPLE',
                 'AWAY_SAMPLE'
                 ]]

In [355]:
table = pd.pivot_table(df_sum2, index=['F_LINE_ID','F_LINE','L_LINE'], columns='NO_REPEAT_PATTERN', 
                       values=['HOME_WIN_RATIO','AWAY_WIN_RATIO','HOME_SAMPLE','AWAY_SAMPLE'],fill_value=0)

In [356]:
table.reset_index(col_level=1).to_csv('output.csv',index=False)

# Sample Check

In [291]:
df_pat = df.copy()
#number_repeating_pattern = 5
#pattern = r'0-{' + str(number_repeating_pattern) + r'}'
pattern = r'0-{5,}$'
df_pat['A_PAT_FOUND'] = df_pat.AWAY_TREND.apply(lambda x: 'Y' if re.findall(pattern,x) else 'N')
df_pat['H_PAT_FOUND'] = df_pat.HOME_TREND.apply(lambda x: 'Y' if re.findall(pattern,x) else 'N')
df_pat['PAT_FOUND'] = df_pat.apply(lambda rec: 'Y' if ((rec['A_PAT_FOUND'] =='Y') or (rec['H_PAT_FOUND']=='Y')) else 'N',axis=1)

# Generate Summary
df_summary = df_pat[df_pat['PAT_FOUND'] == 'Y']

# Generate Summary
df_found_init = df_summary.copy()
#df_found_init

In [298]:
ivd = {v: k for k, v in bh.line_mapping.items()}

In [300]:
print(ivd,)


{0: '(+2/+2.5)', 1: '(+2)', 2: '(+1.5/+2)', 3: '(+1.5)', 4: '(+1/+1.5)', 5: '(+1)', 6: '(+0.5/+1)', 7: '(+0.5)', 8: '(0/+0.5)', 9: '(0)', 10: '(0/-0.5)', 11: '(-0.5)', 12: '(-0.5/-1)', 13: '(-1)', 14: '(-1/-1.5)', 15: '(-1.5)', 16: '(-1.5/-2)', 17: '(-2)', 18: '(-2/-2.5)', 19: '(-2.5)', 20: '(-2.5/-3)'}


In [296]:
84/(84+64)

0.5675675675675675

In [303]:
line = ivd[13]
line

'(-1)'

In [305]:
df_check = df_found_init[(df_found_init.FIRST_LINE == line) & 
           (df_found_init.LAST_LINE == line) &
           (df_found_init.H_PAT_FOUND =='Y')]
last_num = 10
if not df_check.empty:
    print(df_check.groupby('ID').apply(lambda x: x.iloc[0]).iloc[-last_num:].groupby('HANDI_LABEL').size())
else:
    print("No record")
    #for i in df_check.ID.unique()[-10:]:
    
for i in sorted(df_check.ID.unique())[-last_num:]:    
    print('http://www.hkjcodds.com/match_board/'+str(i), df_check[df_check.ID == i].iloc[0].HANDI_LABEL )

HANDI_LABEL
A    2
D    5
H    3
dtype: int64
http://www.hkjcodds.com/match_board/112318 H
http://www.hkjcodds.com/match_board/112824 H
http://www.hkjcodds.com/match_board/113589 D
http://www.hkjcodds.com/match_board/113605 A
http://www.hkjcodds.com/match_board/113623 D
http://www.hkjcodds.com/match_board/113838 A
http://www.hkjcodds.com/match_board/114595 D
http://www.hkjcodds.com/match_board/115345 D
http://www.hkjcodds.com/match_board/116283 D
http://www.hkjcodds.com/match_board/116351 H


In [147]:
df_check.groupby('ID').apply(lambda x:x.iloc[0]).reset_index(drop=True).iloc[-10:]

,AWAY,AWAY_CHANGE,AWAY_LAG,AWAY_SCORE,AWAY_TREND,DELTA,FIRST_LINE,HA,HANDI_LABEL,HOME,...,MATCH_RESULT,MATCH_TIME,O_FIRST_LINE,O_LINE,O_LINE_LEAD,TIME,TITLE,A_PAT_FOUND,H_PAT_FOUND,PAT_FOUND
161,1.92,0,NaN,1,0++-++,NaT,(0/-0.5),"(1.85, 1.92)",H,1.85,...,(0 : 1) 2 : 1,2017-08-30 07:00:00,10,10,NaN,2017-08-28 19:31:00,古里高聯 VS 柏利斯天奴,N,Y,Y
162,1.93,0,NaN,1,0+++++,NaT,(0/-0.5),"(1.91, 1.93)",H,1.91,...,(1 : 0) 2 : 1,2017-09-02 22:00:00,10,10,NaN,2017-08-31 19:34:00,華素爾 VS 普利茅夫,N,Y,Y
163,1.94,0,NaN,1,0++++,NaT,(0/-0.5),"(1.9, 1.94)",H,1.90,...,(1 : 0) 2 : 1,2017-09-03 17:00:00,10,10,NaN,2017-08-31 19:34:00,FC岐阜 VS FC愛媛,N,Y,Y
164,1.86,0,NaN,0,0+++++,NaT,(0/-0.5),"(1.98, 1.86)",H,1.98,...,(1 : 0) 1 : 0,2017-09-10 20:30:00,10,10,NaN,2017-09-07 19:34:00,般尼 VS 水晶宮,N,Y,Y
165,1.80,0,NaN,1,0++++++++++,NaT,(0/-0.5),"(2.05, 1.8)",H,2.05,...,(2 : 1) 3 : 1,2017-09-14 02:45:00,10,10,NaN,2017-09-11 12:02:00,熱刺 VS 多蒙特,N,Y,Y
166,1.91,0,NaN,1,0+++++,NaT,(0/-0.5),"(1.93, 1.91)",H,1.93,...,(0 : 0) 2 : 1,2017-09-16 03:00:00,10,10,NaN,2017-09-14 12:03:00,般尼茅夫 VS 白禮頓,N,Y,Y
167,1.93,0,NaN,1,0++++++,NaT,(0/-0.5),"(1.91, 1.93)",H,1.91,...,(2 : 1) 2 : 1,2017-09-19 02:00:00,10,10,NaN,2017-09-15 19:44:00,樸迪莫倫斯 VS 費倫斯,N,Y,Y
168,1.78,0,NaN,0,0++++++,NaT,(0/-0.5),"(2.07, 1.78)",H,2.07,...,(0 : 0) 1 : 0,2017-09-20 03:00:00,10,10,NaN,2017-09-18 19:35:00,巴黎FC VS 奧連斯,N,Y,Y
169,1.78,0,NaN,4,0++++++,NaT,(0/-0.5),"(2.0, 1.78)",A,2.00,...,(0 : 2) 1 : 4,2017-09-28 10:30:00,10,10,NaN,2017-09-25 19:34:00,聖荷西地震 VS 芝加哥火燄,N,Y,Y
170,2.02,0,NaN,2,0++++,NaT,(0/-0.5),"(1.82, 2.02)",H,1.82,...,(1 : 1) 3 : 2,2017-10-01 01:45:00,10,10,NaN,2017-09-28 19:42:00,施禾利 VS 格羅寧根,N,Y,Y


In [144]:
df_check.groupby('ID').apply(lambda x:x.iloc[0]).reset_index(drop=True).iloc[-10:].groupby('HANDI_LABEL').size()

HANDI_LABEL
A    1
H    9
dtype: int64

In [32]:
df_summary[(df_summary.FIRST_LINE == '(0/+0.5)') & 
           (df_summary.LAST_LINE == '(0/+0.5)') &
           (df_summary.A_PAT_FOUND =='Y') &
           (df_summary.HOME_SCORE == df_summary.AWAY_SCORE)
          ].groupby(['ID','HANDI_LABEL']).size()

ID      HANDI_LABEL
102390  H              12
104364  H              10
107615  H               9
116010  H              20
dtype: int64

In [27]:
8/(8+6)

0.5714285714285714

In [30]:
8/(8+6+2)

0.5

In [38]:
df_summary[(df_summary.FIRST_LINE == '(0/+0.5)') & 
           (df_summary.LAST_LINE == '(+0.5/+1)') &
           (df_summary.A_PAT_FOUND =='Y') &
           (df_summary.HOME_SCORE != df_summary.AWAY_SCORE)
          ].groupby(['HANDI_LABEL']).size()

HANDI_LABEL
A    47
H    57
dtype: int64

In [49]:
df_summary[(df_summary.FIRST_LINE == '(-1/-1.5)') & 
           (df_summary.LAST_LINE == '(-1/-1.5))') &
           (df_summary.H_PAT_FOUND =='Y')
          ].groupby(['HANDI_LABEL']).size()

Series([], dtype: int64)